<a href="https://colab.research.google.com/github/edvar82/Desafio_Alura/blob/main/Desafio_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Instalação do pacote poppler-utils para manipulação de arquivos PDF e da lib para tratar pdfs

In [ ]:
!apt-get install poppler-utils

In [ ]:
!pip install pdf2image

#Baixando os pdf's de exemplo

In [ ]:
!git clone https://github.com/edvar82/Upload_pdfs.git

# Algorítmo para fazer a extração de dados e salvar a planilha


In [ ]:
import os
import pandas as pd
from pdf2image import convert_from_path
import google.generativeai as genai
from google.colab import userdata
import time
from PIL import Image

API_KEY = userdata.get('API_KEY')
genai.configure(api_key=API_KEY)

def listar_pdfs(caminho_raiz):
    for pasta, subpastas, arquivos in os.walk(caminho_raiz):
        for arquivo in arquivos:
            if arquivo.endswith('.pdf'):
                caminho_completo = os.path.join(pasta, arquivo)
                yield caminho_completo

def extract_info_from_text(organized_text):
    campos_desejados = [
        'Nome', 'CPF', 'Ano/Série', 'Turma',
        'Nome do Responsável', 'CPF do Responsável',
        'Cidade e Data da Assinatura', 'Box marcado', 'NIS'
    ]
    extracted_info = {campo: '' for campo in campos_desejados}

    lines = organized_text.split("\n")

    for line in lines:
        if line.strip():
            parts = line.split(":", 1)
            if len(parts) == 2:
                key, value = parts[0].strip(), parts[1].strip()
                if key in campos_desejados:
                    extracted_info[key] = value

    return extracted_info

def salvar_xlsx(df, diretorio, nome_escola):
    if not df.empty:
        df = df.dropna(how='all')
        xlsx_file_path = os.path.join(diretorio, f'{nome_escola}.xlsx')
        df.to_excel(xlsx_file_path, index=False)
        print("Informações extraídas da escola", nome_escola, "salvas em:", xlsx_file_path)
    else:
        print("Nenhum PDF encontrado na pasta.")

prompt = """
Extraia as seguintes informações do texto:

- Nome:
- CPF:
- Ano/Série:
- Turma:
- Nome do Responsável:
- CPF do Responsável:
- Cidade e Data da Assinatura:
- Box marcado:
- NIS:

Forneça as informações no seguinte formato:

Nome: [Nome completo]
CPF: [Número de CPF]
Ano/Série: [Ano ou série]
Turma: [Número da turma]
Nome do Responsável: [Nome completo do responsável]
CPF do Responsável: [CPF do responsável]
Cidade e Data da Assinatura: [Cidade], [Data no formato: DD de MMMM de AAAA]
Box marcado: [Descrição do que está marcado no box]
NIS: [Número do NIS]

Forneça cada informação em uma linha separada.
"""

modelText = genai.GenerativeModel('gemini-pro')
modelVision = genai.GenerativeModel('gemini-pro-vision')
df_concatenado = pd.DataFrame()

pdf_found = False

for caminho_arquivo in listar_pdfs("/content/Upload_pdfs"):

    partes_caminho = caminho_arquivo.split(os.sep)

    diretorio_escola = os.path.dirname(caminho_arquivo)
    nome_escola = os.path.basename(diretorio_escola)

    print(diretorio_escola, nome_escola)

    escola_anterior = nome_escola
    diretorio_escola_anterior = diretorio_escola

    pdf_found = True

    print(f"Lendo arquivo: {caminho_arquivo}...")
    imagens = convert_from_path(caminho_arquivo)

    partes_caminho = caminho_arquivo.split(os.sep)
    escola = partes_caminho[-2]

    for page_num, imagem in enumerate(imagens):
        try:
            responseImage =modelVision.generate_content([
                                                        'Extraia as seguintes informações do texto:',
                                                        '- Nome:',
                                                        '- CPF:',
                                                        '- Ano/Série:',
                                                        '- Turma:',
                                                        '- Nome do Responsável:',
                                                        '- CPF do Responsável:',
                                                        '- Cidade e Data da Assinatura:',
                                                        '- Box marcado:',
                                                        '- NIS:',
                                                        'Forneça as informações no seguinte formato:',
                                                        'Nome: [Nome completo]',
                                                        'CPF: [Número de CPF]',
                                                        'Ano/Série: [Ano ou série]',
                                                        'Turma: [Número da turma]',
                                                        'Nome do Responsável: [Nome completo do responsável]',
                                                        'CPF do Responsável: [CPF do responsável]',
                                                        'Cidade e Data da Assinatura: [Cidade], [Data no formato: DD de MMMM de AAAA]',
                                                        'Box marcado: [Descrição do que está marcado no box]',
                                                        'NIS: [Número do NIS]',
                                                        'Forneça cada informação em uma linha separada.',
                                                        'Dados que não podem faltar: NIS e CPF. Preciso dessas informações em todas as linhas.',
                                                        imagem
                                                    ])

            response_parts = responseImage.parts
            print(response_parts)
            if response_parts:
                text_parts = [part.text for part in response_parts if part.text]
                print(text_parts)
                response_text = '\n'.join(text_parts)

                response = modelText.generate_content([prompt, response_text])

                extracted_info = extract_info_from_text(response.text)

                df = pd.DataFrame([extracted_info])

                df['Unidade escolar'] = escola

                df_concatenado = pd.concat([df_concatenado, df], ignore_index=True)

            else:
                print("A resposta não contém partes válidas.")

        except Exception as e:
            print(f"Algo deu errado. Erro: {e}")

if pdf_found:
    salvar_xlsx(df_concatenado, diretorio_escola_anterior, escola_anterior)
else:
    print("Nenhum PDF encontrado na pasta.")


Resultado a partir dos pdf's gerados: [Resultado](https://docs.google.com/spreadsheets/d/1WM4lLUmeO1W0njkc4IqfxaYFcKRVenACPuBSza9R36M/edit?usp=sharing)